In [33]:
from osgeo import gdal
import tkinter as tk
from tkinter import filedialog, Toplevel, Label
from PIL import Image, ImageTk
import numpy as np

In [35]:
def read_and_display_tif(file_path):
    # Open the TIF image
    dataset = gdal.Open(file_path)
    
    if dataset is None:
        print(f"Unable to open file: {file_path}")
        return

In [40]:
# Mock function for reading and displaying TIF (replace with actual implementation)
def read_and_display_tif(file_path):
    try:
        img = Image.open(file_path)
        img = img.resize((500, 500))  # Resize to fit the Tkinter window
        img_tk = ImageTk.PhotoImage(img)
        panel.config(image=img_tk)
        panel.image = img_tk

        result_label.config(text=f"File path: {file_path}")
    except Exception as e:
        print(f"Error reading TIF file: {e}")

def select_image():
    file_path = filedialog.askopenfilename(filetypes=[("TIF Files", "*.tif")])
    if file_path:
        read_and_display_tif(file_path)

def process_data():
    # Mock data processing (replace with actual processing logic)
    print("Processing data...")

    # Display "Please Wait" popup
    please_wait_popup = Toplevel(root)
    please_wait_popup.title("Processing")
    please_wait_label = Label(please_wait_popup, text="Please wait...")
    please_wait_label.pack(padx=10, pady=10)

    # Simulate data processing
    root.after(2000, lambda: finish_processing(please_wait_popup))

def finish_processing(popup):
    # Close the popup
    popup.destroy()

    # Simulate processed data (for now just creating an inverted image)
    # Replace this with actual data processing logic
    processed_data = np.invert(np.array(panel.image))  # Example: Invert the displayed image
    img = Image.fromarray(processed_data)
    img = img.resize((500, 500))  # Resize to fit the Tkinter window
    img_tk = ImageTk.PhotoImage(img)
    panel.config(image=img_tk)
    panel.image = img_tk

    result_label.config(text="Processing complete")

# Create the main window
root = tk.Tk()
root.title("NavySeal")

# Create and place a label to display the selected image
panel = tk.Label(root)
panel.pack(padx=10, pady=10)

# Create and place a button to select an image
btn_select = tk.Button(root, text="Select Image", command=select_image)
btn_select.pack(pady=10)

# Create and place a button to process the image
btn_process = tk.Button(root, text="Process Data", command=process_data)
btn_process.pack(pady=10)

# Create and place a label to display the file path or prediction result
result_label = tk.Label(root, text="File path: ")
result_label.pack(pady=10)

# Enter the main loop
root.mainloop()

Processing data...


Exception in Tkinter callback
Traceback (most recent call last):
  File "D:\E_2024_P6\P6\ACT\Software\envs\Conda\lib\tkinter\__init__.py", line 1892, in __call__
    return self.func(*args)
  File "D:\E_2024_P6\P6\ACT\Software\envs\Conda\lib\tkinter\__init__.py", line 814, in callit
    func(*args)
  File "C:\Users\DELL\AppData\Local\Temp\ipykernel_50736\1021627471.py", line 35, in <lambda>
    root.after(2000, lambda: finish_processing(please_wait_popup))
  File "C:\Users\DELL\AppData\Local\Temp\ipykernel_50736\1021627471.py", line 43, in finish_processing
    processed_data = np.invert(np.array(panel.image))  # Example: Invert the displayed image
TypeError: bad operand type for unary ~: 'PhotoImage'


Processing data...


Exception in Tkinter callback
Traceback (most recent call last):
  File "D:\E_2024_P6\P6\ACT\Software\envs\Conda\lib\tkinter\__init__.py", line 1892, in __call__
    return self.func(*args)
  File "D:\E_2024_P6\P6\ACT\Software\envs\Conda\lib\tkinter\__init__.py", line 814, in callit
    func(*args)
  File "C:\Users\DELL\AppData\Local\Temp\ipykernel_50736\1021627471.py", line 35, in <lambda>
    root.after(2000, lambda: finish_processing(please_wait_popup))
  File "C:\Users\DELL\AppData\Local\Temp\ipykernel_50736\1021627471.py", line 43, in finish_processing
    processed_data = np.invert(np.array(panel.image))  # Example: Invert the displayed image
TypeError: bad operand type for unary ~: 'PhotoImage'


Processing data...


Exception in Tkinter callback
Traceback (most recent call last):
  File "D:\E_2024_P6\P6\ACT\Software\envs\Conda\lib\tkinter\__init__.py", line 1892, in __call__
    return self.func(*args)
  File "D:\E_2024_P6\P6\ACT\Software\envs\Conda\lib\tkinter\__init__.py", line 814, in callit
    func(*args)
  File "C:\Users\DELL\AppData\Local\Temp\ipykernel_50736\1021627471.py", line 35, in <lambda>
    root.after(2000, lambda: finish_processing(please_wait_popup))
  File "C:\Users\DELL\AppData\Local\Temp\ipykernel_50736\1021627471.py", line 43, in finish_processing
    processed_data = np.invert(np.array(panel.image))  # Example: Invert the displayed image
TypeError: bad operand type for unary ~: 'PhotoImage'
